# **NETTOYAGE DE LA BASE DE DONNÉE DES CLIENTS**

## **1- Parsing des contacts téléphoniques depuis le fichier vCard**

Traitement des contacts téléphoniques (suppressions des caractères spéciaux, des doublons...)

In [1]:
from pathlib import Path
import re
import pandas as pd
import unicodedata

In [2]:
# Dossier racine du projet
BASE_DIR = Path('..')

# Dossiers
RAW_CLIENTS_DIR = BASE_DIR / 'data' / 'raw' / 'Clients'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'

# Fichiers
VCF_FILE = RAW_CLIENTS_DIR / 'contact.vcf'
CSV_FILE = PROCESSED_DIR / 'contacts.csv'

# Vérification
print(f"VCF existe : {VCF_FILE.exists()}")
print(f"Répertoire de sortie existe : {PROCESSED_DIR.exists()}")

VCF existe : True
Répertoire de sortie existe : True


In [3]:
# Lire le contenu du fichier
with VCF_FILE.open("r", encoding="utf-8") as file:
    vcf_data = file.read()

# Extraction
vcards = vcf_data.strip().split("BEGIN:VCARD")[1:]
contacts = []

for card in vcards:
    name_match = re.search(r'FN:(.+)', card)
    tel_match = re.search(r'TEL[^:]*:(.+)', card)
    if name_match and tel_match:
        name = name_match.group(1).strip()
        phone = tel_match.group(1).strip().replace(" ", "")
        contacts.append((name, phone))

# Création du DataFrame
df = pd.DataFrame(contacts, columns=["Nom", "Téléphone"])
df = df.drop_duplicates()

In [4]:
# Fonction pour retirer les accents
def enlever_accents(texte):
    if not isinstance(texte, str):
        return texte
    texte = texte.replace("œ", "oe").replace("Œ", "Oe")
    texte = unicodedata.normalize('NFKD', texte)
    return ''.join([c for c in texte if not unicodedata.combining(c)])

def nettoyer_nom(nom):
    if not nom or nom.strip() == "":
        return "(none)"

    # Cas particuliers invalides
    if any(x in nom for x in ["Kərij Client", "+237", "... Client"]):
        return "(none)"

    # Supprimer les titres (M, Me, R, Dr, DG, etc.)
    titres = r'^(M(?:e)?\.?|Mr\.?|Dr\.?|Mme\.?|Madame|Me|Monsieur|DG\.?|Pr\.?|Ministre\.?|R)\b[\s\.]*'
    nom = re.sub(titres, '', nom, flags=re.IGNORECASE)

    # Supprimer les caractères non-alphanumériques sauf tiret/espace
    nom = re.sub(r'[^\w\s\-]', '', nom)

    # Supprimer les numéros infiltrés
    nom = re.sub(r'\b237\d{8,9}\b', '', nom)

    # Nettoyage final
    nom = ' '.join(nom.split())
    return nom.title() if nom else "(none)"


# Fonction de formatage des numéros
def format_numero(numero):
    numero = numero.replace(" ", "").replace("+", "")
    if numero.startswith("237"):
        reste = numero[3:]
        if not reste.startswith("6") and len(reste) in [8, 9]:
            reste = "6" + reste
        return "+237" + reste[:9]
    if numero.startswith("6") and len(numero) >= 8:
        return "+237" + numero[:9]
    return "+" + numero if not numero.startswith("+") else numero


In [5]:
# Nettoyage des noms et des téléphones
df['Nom'] = df['Nom'].apply(lambda x: nettoyer_nom(str(x)) if not str(x).isdigit() else "(none)")
df['Téléphone'] = df['Téléphone'].apply(format_numero)

# Suppression des doublons
df = df.drop_duplicates()

# Tri alphabétique par nom
df = df.sort_values(by='Nom', key=lambda x: x.str.lower().map(enlever_accents)).reset_index(drop=True)

In [6]:
# Affichage infos
print(f"\nNombre total de contacts uniques après nettoyage : {len(df)}")


Nombre total de contacts uniques après nettoyage : 2103


In [7]:
# Visualisation
df.head()

,Nom,Téléphone
0,(none),+237680160775
1,(none),+237697591972
2,(none),+237699784663
3,(none),+237693573749
4,(none),+237699362631


In [8]:
df.tail()

,Nom,Téléphone
2098,Zita Bbs,+237656479727
2099,Zita Bbs 2,+237656283664
2100,Zoa Commercial,+237693964689
2101,Zoboussi,+237675845542
2102,Zoni,+237677845220


In [9]:
# Sauvegarde
df.to_csv(CSV_FILE, index=False)
print(f"\n✅ Fichier enregistré : {CSV_FILE}")


✅ Fichier enregistré : ../data/processed/contacts.csv


## **2- Consolidation des fichiers clients**

In [10]:
import random
import string
from collections import OrderedDict

# Configuration des chemins
BASE_DIR = Path('..')
RAW_CLIENTS_DIR = BASE_DIR / 'data' / 'raw' / 'Clients'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'

In [11]:
# Fichiers sources
files = {
    "clients_2022": PROCESSED_DIR / "clients_2022.csv",
    "clients_2023": PROCESSED_DIR / "clients_2023.csv",
    "clients_2024": PROCESSED_DIR / "clients_2024.csv",
    "clients_v1": PROCESSED_DIR / "clients(v1).csv",
    "contacts": PROCESSED_DIR / "contacts.csv",
    "client_machine": RAW_CLIENTS_DIR / "Client(01).csv"
}

def load_client_file(filepath):
    """Charge un fichier client avec gestion d'erreurs"""
    try:
        df = pd.read_csv(filepath)
        print(f"✅ Chargé : {filepath.name} ({df.shape[0]} lignes, {df.shape[1]} colonnes)")
        print(f"   Colonnes : {list(df.columns)}")
        return df
    except Exception as e:
        print(f"❌ Erreur chargement {filepath.name}: {e}")
        return pd.DataFrame()
    
# Chargement des fichiers
print("=== CHARGEMENT DES FICHIERS ===")
df_22 = load_client_file(files["clients_2022"])
df_23 = load_client_file(files["clients_2023"])
df_24 = load_client_file(files["clients_2024"])
df_v1 = load_client_file(files["clients_v1"])
df_contacts = load_client_file(files["contacts"])

# Chargement fichier machine avec séparateur spécifique
try:
    df_machine = pd.read_csv(files["client_machine"], sep=";", names=["nom", "telephone"])
    print(f"✅ Chargé : {files['client_machine'].name} ({df_machine.shape[0]} lignes)")
except Exception as e:
    print(f"❌ Erreur chargement machine: {e}")
    df_machine = pd.DataFrame()

=== CHARGEMENT DES FICHIERS ===
✅ Chargé : clients_2022.csv (84 lignes, 8 colonnes)
   Colonnes : ['nom', 'alias', 'telephone', 'telephone_secondaire', 'commande', 'adresse', 'ville', 'nb_commandes']
✅ Chargé : clients_2023.csv (180 lignes, 8 colonnes)
   Colonnes : ['nom', 'alias', 'telephone', 'telephone_secondaire', 'commande', 'adresse', 'ville', 'nb_commandes']
✅ Chargé : clients_2024.csv (123 lignes, 8 colonnes)
   Colonnes : ['nom', 'alias', 'telephone', 'telephone_secondaire', 'commande', 'adresse', 'ville', 'nb_commandes']
✅ Chargé : clients(v1).csv (101 lignes, 6 colonnes)
   Colonnes : ['client_id', 'nom', 'numero', 'nb_proformas', 'ville', 'alias']
✅ Chargé : contacts.csv (2103 lignes, 2 colonnes)
   Colonnes : ['Nom', 'Téléphone']
✅ Chargé : Client(01).csv (9585 lignes)


In [12]:
# Fonctions de nettoyage
def nettoyer_nom(nom):
    """Nettoie et formate les noms"""
    if pd.isna(nom) or nom == "" or str(nom).strip() == "":
        return "(none)"
    
    nom = str(nom).strip()
    
    # Cas particuliers invalides
    if any(x in nom for x in ["Kərij Client", "+237", "... Client"]):
        return "(none)"
    
    # Correction des noms commençant par 0
    if nom.startswith("0") and len(nom) > 1:
        nom = "O" + nom[1:]
    
    # Supprimer les titres (M, Me, R, Dr, DG, etc.)
    titres = r'^(M(?:e)?\.?|Mr\.?|Dr\.?|Mme\.?|Madame|Me|Monsieur|DG\.?|Pr\.?|Ministre\.?|R)\b[\s\.]*'
    nom = re.sub(titres, '', nom, flags=re.IGNORECASE)
    
    # Supprimer les caractères non-alphanumériques sauf tiret/espace
    nom = re.sub(r'[^\w\s\-]', '', nom)
    
    # Supprimer les numéros infiltrés
    nom = re.sub(r'\b237\d{8,9}\b', '', nom)
    
    # Normalisation Unicode et suppression des accents
    nom = unicodedata.normalize("NFKD", nom).encode("ascii", "ignore").decode("utf-8")
    
    # Nettoyage final
    nom = ' '.join(nom.split()).strip().title()
    
    # Vérifier qu'il reste au moins une lettre
    return nom if nom and any(c.isalpha() for c in nom) else "(none)"

def formater_numero(tel):
    """Formate les numéros de téléphone"""
    if pd.isna(tel) or tel == "":
        return ""
    
    tel = str(tel).replace(" ", "").replace("+", "").replace("-", "").replace("(", "").replace(")", "")
    
    # Supprimer les caractères non-numériques
    tel = re.sub(r'[^\d]', '', tel)
    
    if not tel:
        return ""
    
    # Formatage selon les règles du Cameroun
    if tel.startswith("237") and len(tel) >= 12:
        return "+237" + tel[3:12]
    elif tel.startswith("237") and len(tel) >= 9:
        reste = tel[3:]
        if not reste.startswith("6") and len(reste) in [8, 9]:
            reste = "6" + reste
        return "+237" + reste[:9]
    elif tel.startswith("6") and len(tel) >= 8:
        return "+237" + tel[:9]
    elif len(tel) >= 8 and not tel.startswith("6"):
        return "+237" + "6" + tel[:8]
    
    return ""

In [13]:
# Standardisation des colonnes
colonnes_standard = ["nom", "telephone", "telephone_secondaire", "adresse", "ville", "commande", "nb_commandes", "alias"]

def standardiser_colonnes(df, source_name):
    """Standardise les colonnes d'un DataFrame"""
    print(f"\n--- Standardisation {source_name} ---")
    print(f"Colonnes avant : {list(df.columns)}")
    
    # Mappage des colonnes selon les différents formats
    if source_name == "contacts":
        df = df.rename(columns={"Nom": "nom", "Téléphone": "telephone"})
    
    # Ajouter les colonnes manquantes
    for col in colonnes_standard:
        if col not in df.columns:
            df[col] = ""
    
    # Sélectionner uniquement les colonnes standard
    df = df[colonnes_standard].copy()
    
    print(f"Colonnes après : {list(df.columns)}")
    print(f"Lignes : {len(df)}")
    
    return df

# Standardisation des DataFrames
print("\n=== STANDARDISATION DES COLONNES ===")
df_22_std = standardiser_colonnes(df_22, "clients_2022") if not df_22.empty else pd.DataFrame(columns=colonnes_standard)
df_23_std = standardiser_colonnes(df_23, "clients_2023") if not df_23.empty else pd.DataFrame(columns=colonnes_standard)
df_24_std = standardiser_colonnes(df_24, "clients_2024") if not df_24.empty else pd.DataFrame(columns=colonnes_standard)
df_v1_std = standardiser_colonnes(df_v1, "clients_v1") if not df_v1.empty else pd.DataFrame(columns=colonnes_standard)
df_contacts_std = standardiser_colonnes(df_contacts, "contacts") if not df_contacts.empty else pd.DataFrame(columns=colonnes_standard)

# Traitement du fichier machine
if not df_machine.empty:
    df_machine_std = pd.DataFrame(columns=colonnes_standard)
    df_machine_std["nom"] = df_machine["nom"]
    df_machine_std["telephone"] = df_machine["telephone"]
    df_machine_std = df_machine_std.fillna("")
else:
    df_machine_std = pd.DataFrame(columns=colonnes_standard)


=== STANDARDISATION DES COLONNES ===

--- Standardisation clients_2022 ---
Colonnes avant : ['nom', 'alias', 'telephone', 'telephone_secondaire', 'commande', 'adresse', 'ville', 'nb_commandes']
Colonnes après : ['nom', 'telephone', 'telephone_secondaire', 'adresse', 'ville', 'commande', 'nb_commandes', 'alias']
Lignes : 84

--- Standardisation clients_2023 ---
Colonnes avant : ['nom', 'alias', 'telephone', 'telephone_secondaire', 'commande', 'adresse', 'ville', 'nb_commandes']
Colonnes après : ['nom', 'telephone', 'telephone_secondaire', 'adresse', 'ville', 'commande', 'nb_commandes', 'alias']
Lignes : 180

--- Standardisation clients_2024 ---
Colonnes avant : ['nom', 'alias', 'telephone', 'telephone_secondaire', 'commande', 'adresse', 'ville', 'nb_commandes']
Colonnes après : ['nom', 'telephone', 'telephone_secondaire', 'adresse', 'ville', 'commande', 'nb_commandes', 'alias']
Lignes : 123

--- Standardisation clients_v1 ---
Colonnes avant : ['client_id', 'nom', 'numero', 'nb_proforma

In [14]:
# Concaténation de tous les DataFrames
print("\n=== CONCATÉNATION ===")
df_all_clients = pd.concat([
    df_22_std,
    df_23_std,
    df_24_std,
    df_v1_std,
    df_contacts_std,
    df_machine_std
], ignore_index=True)

print(f"Total lignes avant nettoyage : {len(df_all_clients)}")


=== CONCATÉNATION ===
Total lignes avant nettoyage : 12176


In [15]:
# Nettoyage des données
print("\n=== NETTOYAGE DES DONNÉES ===")
df_all_clients["nom"] = df_all_clients["nom"].apply(nettoyer_nom)
df_all_clients["telephone"] = df_all_clients["telephone"].apply(formater_numero)
df_all_clients["telephone_secondaire"] = df_all_clients["telephone_secondaire"].apply(formater_numero)

# Supprimer les lignes sans numéro de téléphone valide
df_all_clients = df_all_clients[df_all_clients["telephone"] != ""].reset_index(drop=True)
print(f"Lignes après suppression des téléphones invalides : {len(df_all_clients)}")


=== NETTOYAGE DES DONNÉES ===
Lignes après suppression des téléphones invalides : 12024


In [16]:
# Remplir les valeurs manquantes
df_all_clients = df_all_clients.fillna("")

# Groupement par numéro de téléphone
print("\n=== GROUPEMENT PAR NUMÉRO ===")
def consolider_valeurs(serie, separateur=" | "):
    """Consolide les valeurs non-vides d'une série"""
    valeurs = [str(v).strip() for v in serie if pd.notna(v) and str(v).strip() != ""]
    valeurs_uniques = list(OrderedDict.fromkeys(valeurs))
    return separateur.join(valeurs_uniques) if valeurs_uniques else ""

def consolider_telephones_secondaires(serie):
    """Consolide les téléphones secondaires"""
    telephones = [str(v).strip() for v in serie if pd.notna(v) and str(v).strip() != "" and str(v).strip() != "0"]
    telephones_uniques = list(OrderedDict.fromkeys(telephones))
    return ", ".join(telephones_uniques) if telephones_uniques else ""

def consolider_commandes_avec_historique(serie):
    """Consolide les commandes en gardant l'ordre chronologique"""
    commandes = []
    for v in serie:
        if pd.notna(v) and str(v).strip() != "":
            commandes.append(str(v).strip())
    
    if not commandes:
        return ""
    
    # Numéroter les commandes pour créer un historique
    commandes_numerotees = []
    for i, cmd in enumerate(commandes, 1):
        commandes_numerotees.append(f"Cmd{i}: {cmd}")
    
    return " | ".join(commandes_numerotees)

# Groupement avec conservation de l'historique
grouped = df_all_clients.groupby("telephone").agg({
    "nom": lambda noms: list(noms),
    "telephone_secondaire": consolider_telephones_secondaires,
    "adresse": lambda x: consolider_valeurs(x, " | "),
    "ville": lambda x: consolider_valeurs(x, " | "),
    "commande": consolider_commandes_avec_historique,
    "nb_commandes": lambda x: sum(pd.to_numeric(x, errors='coerce').fillna(0).astype(int)),
    "alias": lambda x: consolider_valeurs(x, ", ")
}).reset_index()

print(f"Clients uniques après groupement : {len(grouped)}")


=== GROUPEMENT PAR NUMÉRO ===
Clients uniques après groupement : 11018


In [17]:
# Gestion des noms principaux et alias
def extraire_nom_principal_et_alias(noms_list):
    """Extrait le nom principal et les alias"""
    noms_valides = [n for n in noms_list if n != "(none)" and n.strip() != ""]
    
    if not noms_valides:
        return "(none)", ""
    
    # Le nom principal est le premier nom alphabétiquement (plus stable)
    nom_principal = sorted(set(noms_valides), key=lambda x: x.lower())[0]
    
    # Les alias sont les autres noms
    autres_noms = sorted(set(noms_valides) - {nom_principal})
    alias = ", ".join(autres_noms) if autres_noms else ""
    
    return nom_principal, alias

print("\n=== EXTRACTION NOMS ET ALIAS ===")
noms_et_alias = grouped["nom"].apply(extraire_nom_principal_et_alias)
grouped["nom"] = [x[0] for x in noms_et_alias]
grouped["alias_calcule"] = [x[1] for x in noms_et_alias]


=== EXTRACTION NOMS ET ALIAS ===


In [18]:
# Fusion des alias existants et calculés
def fusionner_alias(alias_existant, alias_calcule):
    """Fusionne les alias existants et calculés"""
    alias_list = []
    
    if alias_existant and alias_existant.strip():
        alias_list.extend([a.strip() for a in alias_existant.split(",")])
    
    if alias_calcule and alias_calcule.strip():
        alias_list.extend([a.strip() for a in alias_calcule.split(",")])
    
    alias_uniques = list(OrderedDict.fromkeys([a for a in alias_list if a]))
    return ", ".join(alias_uniques) if alias_uniques else ""

grouped["alias_final"] = grouped.apply(
    lambda row: fusionner_alias(row["alias"], row["alias_calcule"]), 
    axis=1
)

# Création du DataFrame final
final = grouped[[
    "nom", "telephone", "telephone_secondaire", "adresse", "ville", "commande", "nb_commandes", "alias_final"
]].rename(columns={
    "commande": "historique_commandes",
    "alias_final": "alias"
}).copy()

# Ajout des codes clients uniques
def generer_code_client():
    """Génère un code client unique"""
    return "CLT" + ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

final.insert(0, "code_client", [generer_code_client() for _ in range(len(final))])

# Tri par nom
final = final.sort_values(by="nom", key=lambda x: x.str.lower()).reset_index(drop=True)

In [19]:
# === NORMALISATION DES VILLES ===
normalisation_ville = {
    "dla": "Douala",
    "douala": "Douala",
    "dla.": "Douala",
    "yde": "Yaoundé",
    "yaounde": "Yaoundé",
    "yaoundé": "Yaoundé",
    "yde.": "Yaoundé"
}

final["ville"] = final["ville"].astype(str).str.strip().str.lower().map(normalisation_ville).fillna(final["ville"])

In [20]:
# Fusion des alias existants et calculés
def fusionner_alias(alias_existant, alias_calcule):
    """Fusionne les alias existants et calculés"""
    alias_list = []
    
    if alias_existant and alias_existant.strip():
        alias_list.extend([a.strip() for a in alias_existant.split(",")])
    
    if alias_calcule and alias_calcule.strip():
        alias_list.extend([a.strip() for a in alias_calcule.split(",")])
    
    alias_uniques = list(OrderedDict.fromkeys([a for a in alias_list if a]))
    return ", ".join(alias_uniques) if alias_uniques else ""

grouped["alias_final"] = grouped.apply(
    lambda row: fusionner_alias(row["alias"], row["alias_calcule"]), 
    axis=1
)

# Création du DataFrame final
final = grouped[[
    "nom", "telephone", "telephone_secondaire", "adresse", "ville", "commande", "nb_commandes", "alias_final"
]].rename(columns={
    "commande": "historique_commandes",
    "alias_final": "alias"
}).copy()

# Ajout des codes clients uniques
def generer_code_client():
    """Génère un code client unique"""
    return "CLT" + ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

final.insert(0, "code_client", [generer_code_client() for _ in range(len(final))])

# Tri par nom
final = final.sort_values(by="nom", key=lambda x: x.str.lower()).reset_index(drop=True)

In [21]:
# Statistiques finales
print("\n=== STATISTIQUES FINALES ===")
print(f"\n✅ Total clients uniques : {len(final)}")
print(f"\n✅ Clients avec nom valide : {(final['nom'] != '(none)').sum()}")
print(f"\n✅ Clients avec téléphone secondaire : {(final['telephone_secondaire'] != '').sum()}")
print(f"\n✅ Clients ayant passé commande : {(final['nb_commandes'] > 0).sum()}")
print(f"\n✅ Clients avec historique de commandes : {(final['historique_commandes'] != '').sum()}")
print(f"\n✅ Clients avec alias : {(final['alias'] != '').sum()}")


=== STATISTIQUES FINALES ===

✅ Total clients uniques : 11018

✅ Clients avec nom valide : 9914

✅ Clients avec téléphone secondaire : 11

✅ Clients ayant passé commande : 333

✅ Clients avec historique de commandes : 333

✅ Clients avec alias : 260


In [22]:
print("\n✅ Répartition par ville :")
villes = final["ville"].value_counts().head(10)
for ville, count in villes.items():
    print(f"  {ville}: {count}")


✅ Répartition par ville :
  : 10685
  Yaoundé: 245
  Douala: 86
  Nanga: 1
  Tonga: 1


In [23]:
print("\n✅ Exemples de clients avec téléphones secondaires :")
clients_tel_sec = final[final["telephone_secondaire"] != ""].head(5)
for _, client in clients_tel_sec.iterrows():
    print(f"  {client['nom']}: {client['telephone']} | {client['telephone_secondaire']}")


✅ Exemples de clients avec téléphones secondaires :
  Akon Anne Paule: +237697151583 | +237658641752
  Chantal Fassel: +237677582408 | +237633614249
  Client: +237695329696 | +237691409057
  Denise Ngoumou: +237673265846 | +237696218739
  Deutcha: +237654818407 | +237698221364


In [24]:
print("\n✅ Exemples d'historiques de commandes :")
clients_hist = final[final["historique_commandes"] != ""].head(3)
for _, client in clients_hist.iterrows():
    print(f"  {client['nom']}: {client['historique_commandes'][:100]}...")


✅ Exemples d'historiques de commandes :
  Aandjou Carine: Cmd1: Maternelle 1, 5ème | Cmd2: FOURNITURES...
  Abang Arie: Cmd1: (None), FOURNITURES...
  Abassa: Cmd1: 6ème, Form 2 | Cmd2: 5e/FORM 3...


In [25]:
# Sauvegarde
output_file = PROCESSED_DIR / 'final' /  "clients_final.csv"
final.to_csv(output_file, index=False, encoding='utf-8')
print(f"\n✅ Fichier sauvegardé : {output_file}")


✅ Fichier sauvegardé : ../data/processed/final/clients_final.csv


In [26]:
# Statistiques finales
print("\n=== STATISTIQUES FINALES ===")
print(f"\n✅ Total clients uniques : {len(final)}")
print(f"\n✅ Clients avec nom valide : {(final['nom'] != '(none)').sum()}")
print(f"\n✅ Clients avec téléphone secondaire : {(final['telephone_secondaire'] != '').sum()}")
print(f"\n✅ Clients ayant passé commande : {(final['nb_commandes'] > 0).sum()}")
print(f"\n✅ Clients avec historique de commandes : {(final['historique_commandes'] != '').sum()}")
print(f"\n✅ Clients avec alias : {(final['alias'] != '').sum()}")


=== STATISTIQUES FINALES ===

✅ Total clients uniques : 11018

✅ Clients avec nom valide : 9914

✅ Clients avec téléphone secondaire : 11

✅ Clients ayant passé commande : 333

✅ Clients avec historique de commandes : 333

✅ Clients avec alias : 260


In [27]:
print("\n✅ Répartition par ville :")
villes = final["ville"].value_counts().head(10)
for ville, count in villes.items():
    print(f"  {ville}: {count}")


✅ Répartition par ville :
  : 10685
  Yaoundé: 245
  Douala: 86
  Nanga: 1
  Tonga: 1


In [28]:
print("\n✅ Exemples de clients avec téléphones secondaires :")
clients_tel_sec = final[final["telephone_secondaire"] != ""].head(5)
for _, client in clients_tel_sec.iterrows():
    print(f"  {client['nom']}: {client['telephone']} | {client['telephone_secondaire']}")


✅ Exemples de clients avec téléphones secondaires :
  Akon Anne Paule: +237697151583 | +237658641752
  Chantal Fassel: +237677582408 | +237633614249
  Client: +237695329696 | +237691409057
  Denise Ngoumou: +237673265846 | +237696218739
  Deutcha: +237654818407 | +237698221364


In [29]:
print("\n✅ Exemples d'historiques de commandes :")
clients_hist = final[final["historique_commandes"] != ""].head(3)
for _, client in clients_hist.iterrows():
    print(f"  {client['nom']}: {client['historique_commandes'][:100]}...")


✅ Exemples d'historiques de commandes :
  Aandjou Carine: Cmd1: Maternelle 1, 5ème | Cmd2: FOURNITURES...
  Abang Arie: Cmd1: (None), FOURNITURES...
  Abassa: Cmd1: 6ème, Form 2 | Cmd2: 5e/FORM 3...


In [30]:
# Sauvegarde
output_file = PROCESSED_DIR / 'final' /  "clients_final.csv"
final.to_csv(output_file, index=False, encoding='utf-8')
print(f"\n✅ Fichier sauvegardé : {output_file}")


✅ Fichier sauvegardé : ../data/processed/final/clients_final.csv


In [31]:
# Affichage des premières lignes pour vérification
print("\n=== APERÇU DES DONNÉES FINALES ===")
final.head()


=== APERÇU DES DONNÉES FINALES ===


,code_client,nom,telephone,telephone_secondaire,adresse,ville,historique_commandes,nb_commandes,alias
0,CLTLGOSS,(none),+237696235900,,,,,0,
1,CLT0AZM5,(none),+237691050207,,,,,0,
2,CLTMEIWI,(none),+237691052624,,,,,0,
3,CLTJFVCA,(none),+237691062029,,,,,0,
4,CLTKVCGO,(none),+237698363790,,,,,0,


In [32]:
final.tail()

,code_client,nom,telephone,telephone_secondaire,adresse,ville,historique_commandes,nb_commandes,alias
11013,CLT58QJK,Zorel Melvin,+237698218030,,,,,0,
11014,CLT41P84,Zouliatou,+237699510064,,,,,0,
11015,CLTEOPZC,Zounkaraneni,+237693498789,,,,,0,
11016,CLTRYVTG,Zounkifili,+237697168921,,,,,0,
11017,CLT79MYJ,Zra,+237690549509,,,,,0,
